# Calculate surface ocean heat content using CESM2 LENS data

- This notebook is adapted from the NCAR gallery in the Pangeo collection
- https://gallery.pangeo.io/repos/NCAR/notebook-gallery/notebooks/Run-Anywhere/Ocean-Heat-Content/OHC_tutorial.html

### Input Data Access

- This notebook illustrates how to compute surface ocean heat content using potential temperature data from CESM2 Large Ensemble Dataset (https://www.cesm.ucar.edu/community-projects/lens2) hosted on NCAR's glade storage.
- This data is open access and is accessed via OSDF

In [1]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import re
# import nest_asyncio
# nest_asyncio.apply()
import xesmf as xe
import matplotlib.pyplot as plt

In [2]:
import fsspec.implementations.http as fshttp
from pelicanfs.core import PelicanFileSystem, PelicanMap, OSDFFileSystem 

import cf_units as cf

In [3]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [4]:
init_year0  = '1991'
init_year1  = '2020'
final_year0 = '2071'
final_year1 = '2100'

In [5]:
def to_daily(ds):
    year = ds.time.dt.year
    day = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), day=("time", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "day")).unstack("time")

In [6]:
rda_scratch = '/glade/campaign/collections/rda/scratch/harshah'
rda_data    = '/glade/campaign/collections/rda/data/harshah'
# zarr_path   = rda_scratch + "/tas_zarr/"
# mean_path   = zarr_path + "/means/"
# stdev_path  = zarr_path + "/stdevs/"

## Create a PBS cluster

In [7]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk24-hpc',
    cores = 1,
    memory = '8GiB',
    processes = 1,
    local_directory = rda_scratch+'/dask/spill',
    log_directory = rda_scratch + '/dask/logs/',
    resource_spec = 'select=1:ncpus=1:mem=8GB',
    queue = 'casper',
    walltime = '5:00:00',
    #interface = 'ib0'
    interface = 'ext'
)

In [8]:
cluster.scale(10)

In [9]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/46053/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.93:40321,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/46053/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Load CESM LENS2 temperature data

In [12]:
cesm_cat = intake.open_esm_datastore(rda_data + '/intake_catalogs/posix/aws-cesm2-le.json')
cesm_cat

,unique
Unnamed: 0,322
variable,53
long_name,51
component,4
experiment,2
forcing_variant,2
frequency,3
vertical_levels,3
spatial_domain,3
units,20


In [13]:
# cesm_cat.df['variable'].values

In [14]:
cesm_temp = cesm_cat.search(variable ='TEMP', frequency ='monthly')
cesm_temp

,unique
Unnamed: 0,3
variable,1
long_name,1
component,1
experiment,2
forcing_variant,2
frequency,1
vertical_levels,1
spatial_domain,1
units,1


In [15]:
cesm_temp.df['path'].values

array(['/glade/campaign/collections/rda/transfer/chifan_AWS/ncar-cesm2-lens/ocn/monthly/cesm2LE-historical-cmip6-TEMP.zarr',
       '/glade/campaign/collections/rda/transfer/chifan_AWS/ncar-cesm2-lens/ocn/monthly/cesm2LE-ssp370-cmip6-TEMP.zarr',
       '/glade/campaign/collections/rda/transfer/chifan_AWS/ncar-cesm2-lens/ocn/monthly/cesm2LE-ssp370-smbb-TEMP.zarr'],
      dtype=object)

In [18]:
dsets_cesm = cesm_temp.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


ESMDataSourceError: Failed to load dataset with key='ocn.ssp370.monthly.smbb'
                 You can use `cat['ocn.ssp370.monthly.smbb'].df` to inspect the assets/files for this key.
                 

In [ ]:
cesm_temp.keys()

In [ ]:
historical       = dsets_cesm['ocn.historical.monthly.cmip6']
future_smbb      = dsets_cesm['ocn.ssp370.monthly.smbb']
future_cmip6     = dsets_cesm['ocn.ssp370.monthly.cmip6']

In [ ]:
# %%time
# merge_ds_cmip6 = xr.concat([historical, future_cmip6], dim='time')
# merge_ds_cmip6 = merge_ds_cmip6.dropna(dim='member_id')

In [17]:
historical

NameError: name 'historical' is not defined

#### Change units

In [ ]:
orig_units = cf.Unit(historical.z_t.attrs['units'])
orig_units

In [ ]:
def change_units(ds, variable_str, variable_bounds_str, target_unit_str):
    orig_units = cf.Unit(ds[variable_str].attrs['units'])
    target_units = cf.Unit(target_unit_str)
    variable_in_new_units = xr.apply_ufunc(orig_units.convert, ds[variable_bounds_str], target_units, dask='parallelized', output_dtypes=[ds[variable_bounds_str].dtype])
    return variable_in_new_units

In [ ]:
historical['z_t']

In [ ]:
depth_levels_in_m = change_units(historical, 'z_t', 'z_t', 'm')
hist_temp_in_degK = change_units(historical, 'TEMP', 'TEMP', 'degK')
fut_cmip6_temp_in_degK = change_units(future_cmip6, 'TEMP', 'TEMP', 'degK')
fut_smbb_temp_in_degK = change_units(future_smbb, 'TEMP', 'TEMP', 'degK')
#
hist_temp_in_degK  = hist_temp_in_degK.assign_coords(z_t=("z_t", depth_levels_in_m['z_t'].data))
hist_temp_in_degK["z_t"].attrs["units"] = "m"
hist_temp_in_degK

In [ ]:
depth_levels_in_m.isel(z_t=slice(0, -1))

In [ ]:
#Compute depth level deltas using z_t levels
depth_level_deltas = depth_levels_in_m.isel(z_t=slice(1, None)).values - depth_levels_in_m.isel(z_t=slice(0, -1)).values
# Optionally, if you want to keep it as an xarray DataArray, re-wrap the result
depth_level_deltas = xr.DataArray(depth_level_deltas, dims=["z_t"], coords={"z_t": depth_levels_in_m.z_t.isel(z_t=slice(0, -1))})
depth_level_deltas                                                                                        

# Compute Ocean Heat content for ocean surface
- Ocean surface is considered to be the top 100m
- The formula for this is: $$ H = \rho C \int_0^z T(z) dz $$


Where H is ocean heat content, the value we are trying to calculate,

$\rho$ is the density of sea water, $1026 kg/m^3$  ,

$C$ is the specific heat of sea water, $3990 J/(kg K)$  ,

$z$ is the depth limit of the calculation in meters,

and $T(z)$ is the temperature at each depth in degrees Kelvin.

In [ ]:
def calc_ocean_heat(delta_level, temperature):
    rho = 1026 #kg/m^3
    c_p = 3990 #J/(kg K)
    weighted_temperature = delta_level * temperature
    heat = weighted_temperature.sum(dim="z_t")*rho*c_p
    return heat

In [ ]:
# Remember that the coordinate z_t still has values in cm
hist_temp_ocean_surface = hist_temp_in_degK.where(hist_temp_in_degK['z_t'] < 1e4,drop=True)
hist_temp_ocean_surface

In [ ]:
depth_level_deltas_surface = depth_level_deltas.where(depth_level_deltas['z_t'] <1e4, drop= True)
depth_level_deltas_surface

In [ ]:
hist_ocean_heat = calc_ocean_heat(depth_level_deltas_surface,hist_temp_ocean_surface)
hist_ocean_heat

### Plot Ocean Heat

In [ ]:
%%time
# Jan, 1850 average over all memebers
hist_ocean_avgheat = hist_ocean_heat.mean('member_id')
hist_ocean_avgheat.isel(time=0).plot()

In [ ]:
%%time
#Plot ocean heat for Jan 2014
hist_ocean_avgheat.isel(time=-12).plot()

### Has the surface ocean heat content increased with time for January ? (Due to Global Warming!)

In [ ]:
hist_ocean_avgheat_ano = hist_ocean_avgheat.isel(time=-12) - hist_ocean_avgheat.isel(time=0)

In [ ]:
%%time
hist_ocean_avgheat_ano.plot()